In [1]:
import sys
sys.path.insert(0, '../Triangulation/')
sys.path.insert(0, '../network_datasets')


In [2]:
from Triangulation import *

import torch
import torch.optim as optim



#from matplotlib import pyplot as plt

import torch.nn as nn

from torch.autograd import Variable
from math import atan2,pow,acos
from  Neural_network import *

from torch.autograd.function import Function

%matplotlib qt


In [27]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    angle=np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))*180/pi
    if angle>180:
        angle=angle-360
    return angle




# A fucntion to acquire the 2 points from the braycenter and chosen direction
def get_points(barycenter,direction):
    rotation=np.array([[cos(pi),-sin(pi)],
                         [sin(pi),cos(pi)] ])
    point1=np.array(direction+barycenter)
    direction=np.dot(rotation,direction)
    point2=np.array(direction+barycenter)
    return point1, point2
    

In [28]:
def get_set_nb_of_points(point_coordinates):
    set_of_numbers=set()
    for index,_ in enumerate(point_coordinates):
        set_of_numbers.add(len(point_coordinates[index][0]))
    return set_of_numbers


def get_indices_nb_of_points(set_of_numbers,number_of_points,point_coordinates):
    indices=[]
    if number_of_points not in set_of_numbers:
        return "No such number of points for sample"
    else:
        for index,_ in enumerate(point_coordinates):
            if len(point_coordinates[index][0])==number_of_points:
                indices.append(index)
        return indices
    
def get_polygons_nb_of_points(indices):
    
    pass

def get_edge_lengths(polygon):
    polygon_edge_lengths=np.empty([polygon.shape[0]])
    for index,_ in enumerate(polygon):
        polygon_edge_lengths[index]=np.linalg.norm(polygon[(index+1)%(polygon.shape[0])]-polygon[index])
    return polygon_edge_lengths
    

def extract_lengths_angles(polygon_set,nb_of_points):
    lengths=[]
    angles=[]
    target_edge_length=[]
    for polygons in polygon_set:
        polygon=np.delete(polygons,2*nb_of_points).reshape(nb_of_points,2)
        lengths.append(get_edge_lengths(polygon))
        angles.append(np.array(np.multiply(pi/180,get_polygon_angles(polygon))))
        target_edge_length.append([polygons[2*nb_of_points]])
    data=np.hstack([lengths,angles,target_edge_length])
    return data,lengths,angles
    
def extract_lengths_angles_in_triangle_form(polygon_set,nb_of_points):
    data,lengths,angles=extract_lengths_angles(polygon_set,nb_of_points)
    data_reformed=np.empty([data.shape[0],3*nb_of_points+1])
    for polygon_index,polygon_lengths in enumerate(lengths.copy()):
        data_reformed[polygon_index][0:3*nb_of_points:3]=polygon_lengths[0:nb_of_points]
        data_reformed[polygon_index][1:3*(nb_of_points-1):3]=polygon_lengths[1:nb_of_points]
        data_reformed[polygon_index][3*nb_of_points-2]=polygon_lengths[0]
        data_reformed[polygon_index][2:3*nb_of_points:3]=angles[polygon_index]
        #including target edge length
        data_reformed[polygon_index][-1]=polygon_set[polygon_index][2*nb_of_points]
        
    return data_reformed

def extract_midpoint_directions(points):
    mid_points=[]
    directions=[]
    for points in points:
        for coordinates in points:
            coordinates_reshape=coordinates.reshape(2,2)
            point_A=np.array(coordinates_reshape[0])
            point_B=np.array(coordinates_reshape[1])
            mid_point=(point_A+point_B)/2
            direction=point_B-mid_point
            if direction[0]<0:
                direction=point_A-mid_point
            mid_points.append(mid_point)
            directions.append(direction)
    mid_points=np.array(mid_points)
    directions=np.array(directions)
        
    mid_points_with_directions=np.hstack([mid_points,directions])
    return mid_points_with_directions,mid_points,directions


def extract_barycenter(points,nb_of_points):
        
    barycenters=[]
    
    polygons=points.reshape(len(points),nb_of_points,2)
    for polygon in polygons:
        barycenter=np.array([polygon[:,0].sum()/nb_of_points,polygon[:,1].sum()/nb_of_points])
        barycenter_triangles=[]
        barycenters.append(barycenter)
        for i in range(nb_of_points):
            barycenter_triangles.append(np.array([barycenter,polygon[i],polygon[(i+1)%nb_of_points]]))
    barycenters=np.array(barycenters).reshape(len(points),1,2)
    
    return np.array(barycenter)




def length(v):
    return sqrt(v[0]**2+v[1]**2)
def dot_product(v,w):
    return v[0]*w[0]+v[1]*w[1]
def determinant(v,w):
    return v[0]*w[1]-v[1]*w[0]
def inner_angle(v,w):
    cosx=dot_product(v,w)/(length(v)*length(w))
    rad=acos(cosx) # in radians
    return rad*180/pi # returns degrees
def angle_counterclockwise(A, B):
    inner=inner_angle(A,B)
    det = determinant(A,B)
    if det<0: 
        return 360-inner
    else: 
        return inner



In [29]:

class myLossfunction(Function):
    
    @staticmethod
    def forward(self,output,target):
        self.save_for_backward(output,target) 

                        
       # output=output.view(int(output.size()[0]/2),2)

        #target=target.view(int(target.size()[0]/2),2)
        distance=torch.nn.PairwiseDistance()
        result=distance(output,target)

        result=torch.FloatTensor(result)
        #self.save_for_backward(result)


        return  result 
    
    
    @staticmethod
    def backward(self,grad_output1):
        input1,target=self.saved_variables
        
        print(input1)
        #distance=torch.nn.PairwiseDistance()(input1.view(int(input1.size()[0]/2),2),target.view(int(target.size()[0]/2),2))
        distance=torch.nn.PairwiseDistance()(input1,target)

        grad_output1=(input1-target)/distance

        
        return grad_output1,None
    
    
class myOtherLossfunction(Function):
    
    @staticmethod
    def forward(self,output,target)->Variable:
        
        torch_sum=0
        for i in range(0,output.size()[1],2):
            euclidean_distance=(output[:,i]-target[:,i]).pow(2)+(output[:,i+1]-target[:,i+1]).pow(2)
            torch_sum+=euclidean_distance
        return  torch_sum

def my_torch_loss_function(a,b)->Variable:
    torch_sum=0
    for i in range(0,a.size()[1],2):
        euclidean_distance=torch.sqrt((a[:,i]-b[:,i]).pow(2)+(a[:,i+1]-b[:,i+1]).pow(2))
        torch_sum+=euclidean_distance
    return  torch_sum   
    
    

    
    
def my_torch_loss_function2(a,b,target_edge_length)->Variable:
    torch_sum=0
    for i in range(0,a.size()[1],2):
        euclidean_distance=torch.sqrt((a[:,i]-b[:,i]).pow(2)+(a[:,i+1]-b[:,i+1]).pow(2))
        torch_sum+=euclidean_distance
    
    torch_sum=torch.div(torch_sum,target_edge_length)
    return  torch_sum   
    
    

    
    
def my_torch_loss_function3(a,b,target_edge_length,l_param)->Variable:
    torch_sum=0
    torch_point_sum=0
    for i in range(0,a.size()[1],2):
        euclidean_distance=torch.sqrt((a[:,i]-b[:,i]).pow(2)+(a[:,i+1]-b[:,i+1]).pow(2))
        torch_sum+=euclidean_distance

    for i in range(0,a.size()[1]-3,2):
        distance_between_points=torch.sqrt((a[:,i]-a[:,i+2]).pow(2)+(a[:,i+1]-a[:,i+3]).pow(2))
        target_edge_length_distance=torch.abs((distance_between_points-target_edge_length))
        torch_point_sum+=target_edge_length_distance

        

        
    
    #torch_sum=torch.div(torch_sum,target_edge_length)+l_param*torch_point_sum
    torch_sum=torch.div(torch_sum,target_edge_length)*(1-l_param)+(l_param)*target_edge_length_distance
    #torch_sum=(1-l_param)*torch_sum+(l_param)*torch_point_sum

    

    return  torch_sum   




def sort_points(point_coordinates,nb_of_points):
    polygon=point_coordinates.reshape(len(point_coordinates),nb_of_points,2)
    barycenters=extract_barycenter(point_coordinates,nb_of_points)
    angles=[]
    polygons=point_coordinates.reshape(len(point_coordinates),nb_of_points,2)
    vectors=polygons-barycenters

    for  barycenter_vectors in vectors:
        for vector in barycenter_vectors:
            angles.append(angle_counterclockwise(np.array([1,0]),vector))
                      
    angles=np.array(angles).reshape(len(vectors),nb_of_points,1)
    point_coordinates_with_angles=np.dstack([polygons,angles])
    point_coordinates_sorted=[]
    for points in point_coordinates_with_angles:
        points_sorted=np.array(sorted(points,key=lambda x: x[2]))
        points_sorted=points_sorted[:,0:2]
        point_coordinates_sorted.append(points_sorted.reshape(1,nb_of_points,2))
    return np.array(point_coordinates_sorted)    
    

class quadratic_bspline(interpolate.interp2d):
    ''' Iveride interp2d to include quadratic spline'''
    def __init__(self,*args,**kws):
        try:
            super(quadratic_bspline,self).__init__(*args,**kws)
        
        except ValueError:      
            kx=ky=2
            x=args[0]
            y=args[1]
            z=args[2]
            rectangular_grid = (z.size == len(x) * len(y))
            if rectangular_grid:
                self.tck = scipy.interpolate.fitpack.bisplrep(x, y, z, kx=kx, ky=ky, s=0.0)
            else:
                nx, tx, ny, ty, c, fp, ier = scipy.interpolate.dfitpack.regrid_smth(
                x, y, z, None, None, None, None,
                kx=kx, ky=ky, s=0.0)
                self.tck = (tx[:nx], ty[:ny], c[:(nx - kx - 1) * (ny - ky - 1)],
                        kx, ky)
            self.bounds_error = False
            self.fill_value = None
            self.x, self.y, self.z = [np.array(a, copy=copy) for a in (x, y, z)]

            self.x_min, self.x_max = np.amin(x), np.amax(x)
            self.y_min, self.y_max = np.amin(y), np.amax(y)


    

NameError: name 'interpolate' is not defined

In [6]:
# Using convolution network
# O: output dimension
# I: Input dimensiion
# S: Stride
# P: padding
# w: kernel size
# O=(I-w-2*P)/S+1

# Included batch normalization with dropout layers

nb_of_points_output=1

class Conv_net(nn.Module):
    
    def __init__(self):
        super(Conv_net,self).__init__()
        self.conv1=nn.Conv1d(1,14,kernel_size=14,stride=1)
        self.conv2=nn.Conv1d(14,28,kernel_size=2,stride=1)

        self.bn1=nn.BatchNorm1d(num_features=28*9)
        self.fc1=nn.Linear(28*9,12)
        self.dp_1=nn.Dropout(p=0.5)

        
        self.bn2=nn.BatchNorm1d(num_features=12)
        self.fc2=nn.Linear(12,12)
        self.dp_2=nn.Dropout(p=0.5)

        
        self.bn3=nn.BatchNorm1d(num_features=12)
        self.fc3=nn.Linear(12,12)
        self.dp_3=nn.Dropout(p=0.5)
        
        self.bn4=nn.BatchNorm1d(num_features=12)
        self.fc4=nn.Linear(12,nb_of_points_output*2)
        
    def forward(self,x):
        x=F.relu(F.max_pool1d(self.conv1(x),kernel_size=2,stride=1))
        
        #x=F.relu(F.max_pool1d(self.conv2(x),kernel_size=2,stride=1))
        #x=self.conv2(x)

        x=F.relu(F.max_pool1d(self.conv2(x),kernel_size=2,stride=1))
        x=F.relu(self.fc1(self.bn1(x.view(-1,28*9))))
        x=F.relu(self.fc2( self.dp_1(self.bn2(x))))
        x=F.relu(self.fc3(self.dp_2(self.bn3(x))))

        x=self.fc4(self.bn4(self.dp_3(x)))

        return x

In [7]:
# Another convolution network involving the pairing of lengths and angles through convolution and then 
# fully connectiong in a  linear fashion
# Input : a matrix X where lengths take X/2 and angles X/2 and the target edge length

class alt_conv_net(nn.Module):
    
    def __init__(self,nb_of_filters,nb_of_hidden_nodes,out_dimension,nb_of_edges):
        super(alt_conv_net,self).__init__()
        
        self.nb_of_edges=nb_of_edges
        self.nb_of_filters=nb_of_filters
        
        self.conv1=nn.Sequential(nn.Conv1d(in_channels=1,out_channels=nb_of_filters,stride=1,kernel_size=2),
                                 nn.MaxPool1d(stride=1,kernel_size=2),nn.ReLU(inplace=True))
        self.conv2=nn.Sequential(nn.Conv1d(in_channels=1,out_channels=nb_of_filters,stride=1,kernel_size=2),
                                 nn.MaxPool1d(stride=1,kernel_size=2),nn.ReLU(inplace=True))
        # The linear connections are fixed for 12 polygon example
   
        self.fc=nn.Sequential(  nn.BatchNorm1d(num_features=nb_of_filters*(nb_of_edges-2)+nb_of_filters*(nb_of_edges-2)+1),

                                nn.Linear(nb_of_filters*(nb_of_edges-2)+nb_of_filters*(nb_of_edges-2)+1,nb_of_hidden_nodes),
                                        nn.ReLU(inplace=True),
                               nn.BatchNorm1d(num_features=nb_of_hidden_nodes),
                               nn.Linear(nb_of_hidden_nodes,nb_of_hidden_nodes),
                                        nn.ReLU(inplace=True),
                                nn.BatchNorm1d(num_features=nb_of_hidden_nodes),

                               nn.Linear(nb_of_hidden_nodes,out_dimension) )                                                                        
        
        
        
        
    def forward(self,x):
        
        lenghts=x.narrow(1,0,1).narrow(2,0,self.nb_of_edges)
        angles=x.narrow(1,0,1).narrow(2,self.nb_of_edges,self.nb_of_edges)
        target_edge_length=x.narrow(1,0,1).narrow(2,2*self.nb_of_edges,1).resize(x.size()[0],1)

       
        conv_result1=self.conv1(lenghts)
        conv_result2=self.conv2(angles)        
        
        # reshape the convolution results
        
        conv_result1=conv_result1.view(-1,self.nb_of_filters*(self.nb_of_edges-2))
        conv_result2=conv_result2.view(-1,self.nb_of_filters*(self.nb_of_edges-2))
        
        concat_tensor=torch.cat([conv_result1,conv_result2,target_edge_length],1)
        output=self.fc(concat_tensor)
        return output
        

        
        

In [8]:
# Another convolutional network. The input of the network is : Length[index], Length[index+1], angle[index] e.g the triangles
# formed by connecting the neighbouring edges of the contour.
# The input is convoluted by a kernel of size 3 with a stride of 3.


class triangle_convoluting_net(nn.Module):
    
    def __init__(self,nb_of_filters,nb_of_hidden_nodes,out_dimension,nb_of_edges):
        super(triangle_convoluting_net,self).__init__()
        self.nb_of_filters=nb_of_filters
        self.nb_of_edges=nb_of_edges
        self.conv=nn.Sequential(nn.Conv1d(in_channels=1,out_channels=nb_of_filters,stride=3,kernel_size=3),
                                #nn.Conv1d(in_channels=nb_of_filters,out_channels=nb_of_filters,stride=1,kernel_size=2),
                                #nn.MaxPool1d(stride=1,kernel_size=2),nn.ReLU(inplace=True),
                                nn.Conv1d(in_channels=nb_of_filters,out_channels=nb_of_filters,kernel_size=2,stride=1),
                                nn.MaxPool1d(stride=1,kernel_size=2),nn.ReLU(inplace=True))
       
        # The linear connections are fixed for 12 polygon example
   
        self.fc=nn.Sequential(  nn.BatchNorm1d(num_features= nb_of_filters*(nb_of_edges-2)+1),
                              
                                # 1rst layer
                                nn.Linear(nb_of_filters*(nb_of_edges-2)+1,nb_of_hidden_nodes),
                                        nn.ReLU(inplace=True),
                               nn.BatchNorm1d(num_features=nb_of_hidden_nodes),
                             
                              # 2nd layer
                            #  nn.Linear(nb_of_hidden_nodes,nb_of_hidden_nodes),
                             #     nn.ReLU(inplace=True),
                             #  nn.BatchNorm1d(num_features=nb_of_hidden_nodes),
                                
                              # 3rd layer
                              #nn.Linear(nb_of_hidden_nodes,nb_of_hidden_nodes),
                               #   nn.ReLU(inplace=True),
                              #nn.BatchNorm1d(num_features=nb_of_hidden_nodes),
                              
                            
                              nn.Linear(nb_of_hidden_nodes,out_dimension)
                             
                             
                             
                             )      

    def forward(self,x):
        
        triangles=x.narrow(1,0,1).narrow(2,0,3*self.nb_of_edges)
        target_edge_length=x.narrow(1,0,1).narrow(2,3*self.nb_of_edges,1).resize(x.size()[0],1)
        
        conv_result=self.conv(triangles)
        
        
        
        conv_result=conv_result.view(-1,self.nb_of_filters*(self.nb_of_edges-2))
        #print(conv_result,target_edge_length)

        concat_tensor=torch.cat([conv_result,target_edge_length],1)
        output=self.fc(concat_tensor)
        
        return output

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

    

In [8]:
# Plotting the number of polygons with the number of points inserted

def plot_population_nb_of_points(set_of_points,point_coordinates):
    number_of_polygons=[]
    for numbers in set_of_points:
        indixes=get_indices_nb_of_points(set_of_points,numbers,point_coordinates)
        number_of_polygons.append(len(indixes))

    
    #plt.xticks(np.array(list(set_of_points)))
    #plt.yticks(np.array(number_of_polygons))
    plt.figure(figsize=(24, 24))  
    plt.xticks(list(set_of_points))
    plt.yticks(number_of_polygons)
    plt.xlabel('number of insertion points')
    plt.ylabel('Polygon counts')
    plt.title('Plot bar for number of polygons with number of insertion points (20000 total)')
    plt.bar(list(set_of_points), number_of_polygons, color='g')


In [9]:
def save_point_regression_network(filename,net):
    path=os.path.join('../network_datasets/point_coord_NN',filename)

    with open(path,'wb') as output:
        pickle.dump(net,output)
        
def load_point_regression_network(filename):
    path=os.path.join('../network_datasets/point_coord_NN',filename)
        
    with open(path,'rb') as input:
        net=pickle.load(input)
        
    net.eval()
    return net

In [10]:
# Get dataset of polygons with a specific target edge length 
def get_polygons_target_edge_length(polygons_reshaped,point_coordinates,target_edge_length):
    polygons_reshaped_target_edge_length=polygons_reshaped[polygons_reshaped[:,int(len(polygons_reshaped))]==target_edge_length]

    target_lentgh_indices=np.where(polygons_reshaped[:,int(len(polygons_reshaped))]==target_edge_length)
    point_coordinates_target_edge_length=point_coordinates[target_lentgh_indices]

    point_coordinates=point_coordinates_target_edge_length
    polygons_reshaped=polygons_reshaped_target_edge_length
    
    
# Given a neural network and a test set for this generate more polygons pertubing those that had an error more than tol
def generate_data(net,x_test,tol):
    net=net.cpu()
    x_test=x_test.cpu()
    net.eval()
    predictions=net(x_test).data.numpy()
    target_edge_lengths=x_test[:,16]
    contours=x_test.data.numpy()
    nb_sampling=10
    
    pertubed_contours=[]
    edge_lengths=[]
    inserted_points_delaunay=[]
    set_of_pertubed_contours=set()
    
    
    for index,prediction in enumerate(predictions):
            contour=contours[index]
            contour=np.delete(contour,16)
            contour=contour.reshape(8,2)
            _,real_value=get_extrapoints_target_length_jupyter(contour,target_edge_lengths[index],algorithm='del2d')
            error=np.linalg.norm(prediction-np.array(real_value))

            if error > tol and hash(tuple(contour.reshape(16))) not in set_of_pertubed_contours:
                print("Pertubing contour, " ,contour)
                set_of_pertubed_contours.add(hash(tuple(contour.reshape(16))))
                print(set_of_pertubed_contours)
                for j in range(8):
                    for i in range(0,nb_of_sampling):
                        pertubed_contour=contour.copy()
                        pertubed_contour[j]=np.random.normal(contour[j],0.1)
                        print(contour[j],"changed to ",pertubed_contour[j])
                        print(pertubed_contour,contour)
                        
                        for edge_length in np.linspace(0.1,1,10):
                            print("Examining target edge length ",edge_length,"for ",pertubed_contour)
                            nb_of_points_delaunay,point_coords_delaunay=get_extrapoints_target_length_jupyter(pertubed_contour,edge_length,algorithm='del2d')
                            if nb_of_points_delaunay==1:
                                
                                inserted_points_delaunay.append(point_coords_delaunay)
                                print("Inserting pertubed_contour",pertubed_contour)
                                pertubed_contours.append(pertubed_contour)
                                edge_lengths.append(edge_length)  
                                #print("List includes",pertubed_contours)
                                #plot_contour(pertubed_contour)
                                #plt.scatter(point_coords_delaunay[:,0],point_coords_delaunay[:,1])

    return    pertubed_contours,edge_lengths,inserted_points_delaunay


# Given a neural network and a test set for this generate more polygons pertubing those that had an error more than tol
# and for a specific target edge length
def generate_data_for_target_edge_length(net,x_test,tol,target_edge_length):
    net=net.cpu()
    x_test=x_test.cpu()
    net.eval()
    predictions=net(x_test).data.numpy()
    target_edge_lengths=x_test[:,16]
    contours=x_test.data.numpy()
    nb_sampling=10
    
    pertubed_contours=[]
    edge_lengths=[]
    inserted_points_delaunay=[]
    set_of_pertubed_contours=set()
    
    
    for index,prediction in enumerate(predictions):
            contour=contours[index]
            contour=np.delete(contour,16)
            contour=contour.reshape(8,2)
            _,real_value=get_extrapoints_target_length_jupyter(contour,target_edge_lengths[index],algorithm='del2d')
            error=np.linalg.norm(prediction-np.array(real_value))

            if error > tol and hash(tuple(contour.reshape(16))) not in set_of_pertubed_contours:
                print("Pertubing contour, " ,contour)
                set_of_pertubed_contours.add(hash(tuple(contour.reshape(16))))
                print(set_of_pertubed_contours)
                for j in range(8):
                    for i in range(0,nb_of_sampling):
                        pertubed_contour=contour.copy()
                        pertubed_contour[j]=np.random.normal(contour[j],0.1)
                        print(contour[j],"changed to ",pertubed_contour[j])
                        print(pertubed_contour,contour)
                        
                        print("Examining target edge length ",edge_length,"for ",pertubed_contour)
                        nb_of_points_delaunay,point_coords_delaunay=get_extrapoints_target_length_jupyter(pertubed_contour,target_edge_length,algorithm='del2d')
                        if nb_of_points_delaunay==1:
                                
                            inserted_points_delaunay.append(point_coords_delaunay)
                            print("Inserting pertubed_contour",pertubed_contour)
                            pertubed_contours.append(pertubed_contour)
                            edge_lengths.append(target_edge_length)  
                                #print("List includes",pertubed_contours)
                                #plot_contour(pertubed_contour)
                                #plt.scatter(point_coords_delaunay[:,0],point_coords_delaunay[:,1])

    return    pertubed_contours,edge_lengths,inserted_points_delaunay



In [11]:
# Recovering the data of polygons where a specific number of points was inserted

def reshape_data(polygons,coordinates,numb_of_ins_points,nb_of_points):
    polygons=np.array([i for i in polygons for j in range(10)])
    polygons_reshaped=[]
    for polygon in polygons:
        polygons_reshaped.append(polygon.reshape(2,polygons.shape[1]))

    polygons_reshaped=np.array(polygons_reshaped)
    set_of_points=get_set_nb_of_points(coordinates)        
    indices=get_indices_nb_of_points(set_of_points,nb_of_points,coordinates)
    indices=np.asarray(indices)
    number_of_insertion_points=np.array(numb_of_ins_points)
    polygons_reshaped.resize(len(coordinates),2*polygons.shape[1])

    polygons_reshaped=np.hstack([polygons_reshaped[indices],number_of_insertion_points[indices,1].reshape(len(indices),1) ])
    coordinates=[ coordinates[i][0]for i in indices]
    coordinates=np.array(coordinates)
    coordinates=coordinates.reshape(polygons_reshaped.shape[0],1,2*nb_of_points)
    return polygons_reshaped,coordinates

In [12]:
# Plot a prediction of a random contour based on a trained network (FFN)

def plot_random_prediction_ffn(polygon_edges,net,number_of_points):
    plt.clf()
    for i in range(1000):
        random_contour=apply_procrustes(generate_contour(polygon_edges))
        random_nb_of_points,random_point_coordinated_delaunay=get_extrapoints_target_length_jupyter(random_contour,0.8,algorithm='del2d')
        if random_nb_of_points==number_of_points:
            break
    random_contour_reshaped=random_contour.reshape(1,2*polygon_edges)
    random_contour_with_target=np.hstack([random_contour_reshaped,[[1]]])



    plot_contour(random_contour)
    random_point_coordinated_delaunay=np.array(random_point_coordinated_delaunay)
    random_point_coordinated_delaunay.reshape(number_of_points,2)
    plt.scatter(random_point_coordinated_delaunay[:,0],random_point_coordinated_delaunay[:,1],label='Point location')




    random_x_variable=Variable(torch.from_numpy(random_contour_with_target))
    random_x_variable=random_x_variable.expand(1000,2*polygon_edges+1).type(torch.FloatTensor)
    net=net.cpu()
    random_prediction=net(random_x_variable)
    random_prediction=random_prediction.data[0].numpy()
    random_prediction=random_prediction.reshape(number_of_points,2)
    plt.scatter(random_prediction[:,0],random_prediction[:,1],label=' fnn prediction')
    plt.legend()
    
    
    
# Plot a prediction of a random contour based on a trained network (CNN)

def plot_random_prediction_cnn(polygon_edges,net,number_of_points):
    plt.clf()
    for i in range(1000):
        random_contour=apply_procrustes(generate_contour(polygon_edges))
        random_nb_of_points,random_point_coordinated_delaunay=get_extrapoints_target_length_jupyter(random_contour,0.8,algorithm='del2d')
        if random_nb_of_points==number_of_points:
            break
    random_contour_reshaped=random_contour.reshape(1,2*polygon_edges)
    random_contour_with_target=np.hstack([random_contour_reshaped,[[1]]])
    triangles_with_edges=extract_lengths_angles_in_triangle_form(random_contour_with_target,polygon_edges)



    plot_contour(random_contour)
    random_point_coordinated_delaunay=np.array(random_point_coordinated_delaunay)
    random_point_coordinated_delaunay.reshape(number_of_points,2)
    plt.scatter(random_point_coordinated_delaunay[:,0],random_point_coordinated_delaunay[:,1],label='Point location')




    random_x_variable=Variable(torch.from_numpy(triangles_with_edges))
    print(random_x_variable)
    random_x_variable=random_x_variable.expand(1000,1,3*polygon_edges+1).type(torch.FloatTensor)
    net=net.cpu()
    random_prediction=net(random_x_variable)
    random_prediction=random_prediction.data[0].numpy()
    random_prediction=random_prediction.reshape(number_of_points,2)
    plt.scatter(random_prediction[:,0],random_prediction[:,1],label='cnn prediction')
    plt.legend()

In [13]:
def train_network(net,epochs,optimization,loss_function,variable_x,variable_x_test,variable_y,variable_y_test):
    batch_size=int(variable_x.size()[0]/149 )
    net=net.cuda()

    # Train the network #
    
    net.train()
    for t in range(nb_of_epochs):
        sum_loss=0
        for b in range(0,variable_x.size(0),batch_size):
            out = net(variable_x.narrow(0,b,batch_size)) # input x and predict based on x
            loss=loss_function(out, variable_y.narrow(0,b,batch_size).resize(batch_size,nb_of_points*1)).sum()
            sum_loss+=loss.data[0]       
            optimization.zero_grad()   # clear gradients for next train
            loss.backward()         # backpropagation, compute gradients
            optimization.step()        # apply gradients
            
        if t%10==0: 
            net.eval()
            out_test=net(variable_x_test)  
            test_loss=loss_function(out_test, variable_y_test.resize(len(variable_y_test),2)).sum()
            print("Epoch:",t,"Training Loss:",sum_loss/(variable_x.size(0))," Test loss: ",test_loss.data[0]/(variable_x_test.size(0)))
            net.train()
    
    

In [14]:

def max_element(A):
    r, (c, l) = max(map(lambda t: (t[0], max(enumerate(t[1]), key=lambda v: v[1])), enumerate(A)), key=lambda v: v[1][1])
    return (l, r, c)


# Precision error for two points
def precision_error(real_points,predictions):
    distances=np.empty([2,2])
    for point_index,point in enumerate(real_points):
        for prediction_index,prediction in enumerate(predictions):
               distances[point_index][prediction_index]=np.linalg.norm(point-prediction)
    
    max_distance,prediction_index,point_index=max_element(distances)
    distance1=distances[prediction_index][(point_index+1)%2]
    distance2=distances[(prediction_index+1)%2][point_index]
    return max(distance2,distance1)



# Precision error for multiple points

def regression_error(real_points,prediction_points,nb_of_interior_points):
    
    real_point_polygon=sort_points(real_points.reshape(1,1,nb_of_interior_points,2),nb_of_interior_points).reshape(nb_of_interior_points,2)
    prediction_points_polygon=sort_points(prediction_points.reshape(1,1,nb_of_interior_points,2),nb_of_interior_points).reshape(nb_of_interior_points,2)
    procrusted_prediction_points_polygon=apply_procrustes(prediction_points_polygon,real_point_polygon)
    
    error_in_transformation=np.array([np.linalg.norm(prediction_points_polygon[indices]-procrusted_prediction_points_polygon[indices]) for indices in range(len(prediction_points_polygon))])

    maximum_index,maximum_distance=np.argmax([np.linalg.norm(real_point_polygon[index]-procrusted_prediction_points_polygon[index]) for index in range(len(real_point_polygon))]),np.max([np.linalg.norm(real_point_polygon[index]-procrusted_prediction_points_polygon[index]) for index in range(len(real_point_polygon))])
    regression_error=maximum_distance+error_in_transformation[maximum_index]
    
    return regression_error    


In [16]:
polygons_initial=load_dataset('6_polygons.pkl')
point_coordinates_initial=load_dataset('6_point_coordinates_del.pkl')
number_of_insertion_points=load_dataset('6_nb_of_points_del.pkl')

In [17]:
nb_of_points=3
polygons_reshaped,point_coordinates=reshape_data(polygons_initial,point_coordinates_initial,number_of_insertion_points,nb_of_points)


In [18]:
polygons_reshaped.shape,point_coordinates.shape



((27765, 13), (27765, 1, 6))

In [19]:
# Shuffle the data

polygons_reshaped,point_coordinates=unison_shuffled_copies(polygons_reshaped,point_coordinates)
point_coordinates=sort_points(point_coordinates,nb_of_points)


NameError: name 'unison_shuffled_copies' is not defined

In [20]:
# 80/20 training/test data ratio

nb_of_test_data=int(len(polygons_reshaped)*0.2)
nb_of_training_data=int(len(polygons_reshaped)-nb_of_test_data)
nb_of_test_data,nb_of_training_data
point_coordinates=point_coordinates.reshape(len(point_coordinates),1,2*nb_of_points)

In [21]:
#mid_points_with_directions,midpoints,directions=extract_midpoint_directions(point_coordinates)
#mid_points_with_directions=mid_points_with_directions.reshape(len(mid_points_with_directions),1,4)
#midpoints=midpoints.reshape(len(midpoints),1,2)
#directions=directions.reshape(len(directions),1,2)




x_tensor=torch.from_numpy(polygons_reshaped[:nb_of_training_data]).type(torch.FloatTensor)
#mixed_tensor=torch.cat((lengths_tensor[:nb_of_training_data],angles_tensor[:nb_of_training_data]),1)
#double_mixed_tensor=angles_tensor[:nb_of_training_data]

x_tensor_test=torch.from_numpy(polygons_reshaped[nb_of_training_data:]).type(torch.FloatTensor)
#mixed_tensor_test=torch.cat((lengths_tensor[nb_of_training_data:],angles_tensor[nb_of_training_data:]),1)
#double_mixed_tensor_test=angles_tensor[nb_of_training_data:]




x_variable,x_variable_test=Variable(x_tensor),Variable(x_tensor_test)
#x_variable,x_variable_test=Variable(mixed_tensor),Variable(mixed_tensor_test)
#x_variable,x_variable_test=Variable(double_mixed_tensor),Variable(double_mixed_tensor_test)

y_tensor=torch.from_numpy(point_coordinates[:nb_of_training_data]).type(torch.FloatTensor)
y_tensor_test=torch.from_numpy(point_coordinates[nb_of_training_data:]).type(torch.FloatTensor)

y_variable,y_variable_test=Variable(y_tensor),Variable(y_tensor_test)




# Testing with different networks

#y_tensor_midpoints=torch.from_numpy(midpoints[:nb_of_training_data]).type(torch.FloatTensor)
#y_tensor_midpoints_test=torch.from_numpy(midpoints[nb_of_training_data:]).type(torch.FloatTensor)

#y_tensor_directions=torch.from_numpy(directions[:nb_of_training_data]).type(torch.FloatTensor)
#y_tensor_directions_test=torch.from_numpy(directions[nb_of_training_data:]).type(torch.FloatTensor)


#y_variable_midpoints,y_variable_midpoints_test=Variable(y_tensor_midpoints),Variable(y_tensor_midpoints_test)
#y_variable_directions,y_variable_directions_test=Variable(y_tensor_directions),Variable(y_tensor_directions_test)


#shuffle=torch.randperm(x_variable.shape[0])
#x_variable = x_variable[shuffle]
#y_variable=y_variable[shuffle]

#print(y_variable_midpoints.size(),x_variable.size())


In [24]:
# Plotting the number of polygons in reference with the target edge length

x_variable=x_variable.cpu()
x_numpy=x_variable.data.numpy()
lengths,counts=np.unique(x_numpy[:,2*nb_of_edges+],return_counts=True)


plt.figure(figsize=(14,23))
plt.xticks(lengths)
plt.yticks(counts,rotation=23,fontsize=6)
plt.xlabel('Target Edge Lengths')
plt.ylabel('Counts')
plt.title('Number of polygon and the correspoding target edge length for one point insertion')

plt.bar(list(lengths), list(counts),  0.05,color='r')


SyntaxError: invalid syntax (<ipython-input-24-478c02e4cee9>, line 5)

In [23]:
my_net=Net(x_variable.size()[1],y_variable.size()[2],nb_of_hidden_layers=2, nb_of_hidden_nodes=45,batch_normalization=True)
torch.cuda.empty_cache()
print("Training data length:",x_variable_test.size()[1],y_variable.size()[2])


Training data length: 13 6


In [24]:
optimizer = torch.optim.Adam(my_net.parameters(), lr=1e-4,weight_decay=0.5)
#optimizer = torch.optim.SGD(my_net.parameters(), lr=1e-4,weight_decay=.5,momentum=0.9)
#max_distance=0.6108970818704328
#loss_func =torch.nn.MSELoss(size_average=False) 
#loss_func=myOtherLossfunction()


In [25]:

if  torch.cuda.is_available():
    #loss_func.cuda()
        
    x_variable , y_variable=x_variable.cuda(), y_variable.cuda()
    x_variable_test,y_variable_test= Variable(x_tensor_test.cuda(),volatile=True),Variable(y_tensor_test.cuda(),volatile=True)
    #x_variable_test,y_variable_test= Variable(double_mixed_tensor_test.cuda(),volatile=True),Variable(y_tensor_test.cuda(),volatile=True)

    print("cuda activated")
    
    
    

cuda activated


In [26]:
batch_size=int(x_variable.size()[0])
nb_of_epochs=13000
my_net.cuda()
#my_net.cpu()

# Train the network #
my_net.train()
for t in range(nb_of_epochs):
    sum_loss1=0
    sum_loss2=0
    for b in range(0,x_variable.size(0),batch_size):
        out = my_net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        #loss2= loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        #loss=loss_func.apply(out, y_variable.narrow(0,b,batch_size).resize(batch_size,2))
        #target_edge_lengths=x_variable.narrow(0,b,batch_size).narrow(1,24,1)
        loss1=my_torch_loss_function3(out, y_variable.narrow(0,b,batch_size).resize(batch_size,nb_of_points*2),x_variable.narrow(0,b,batch_size)[:,24],.3).sum().cuda()
        
        #loss1=my_torch_loss_function2(out, y_variable.narrow(0,b,batch_size).resize(batch_size,nb_of_points*2),x_variable.narrow(0,b,batch_size)[:,16]).sum()
        sum_loss1+=loss1.data[0]
        #sum_loss2+=loss2.data[0]

        
        optimizer.zero_grad()   # clear gradients for next train
        loss1.backward()         # backpropagation, compute gradients
        #print(t,loss1.data[0],loss2.data[0])
        optimizer.step()        # apply gradients
        del loss1
        del out
    if t%10==0: 
        my_net.eval()
        out_test=my_net(x_variable_test)   
        
        #test_loss=my_torch_loss_function3(out_test, y_variable_test.resize(len(y_variable_test),nb_of_points*2)).sum()

        test_loss=my_torch_loss_function3(out_test, y_variable_test.resize(y_variable_test.size()[0],nb_of_points*2),x_variable_test[:,24],.3).sum().cuda()
    
        #test_loss=loss_func(out_test,y_variable_test)
        #my_net.train()
       # print("Epoch:",t,"Training Loss:",sum_loss2/(x_variable.size(0)),"Test Loss:",test_loss.data[0]/x_variable_test.size(0))
        print("Epoch:",t,"Training Loss:",sum_loss1/(x_variable.size(0)),test_loss.data[0]/(x_variable_test.size(0)))
        my_net.train()


IndexError: index 24 is out of range for dimension 1 (of size 9)

In [ ]:
my_net_midpoint=Net(x_variable.size()[1],y_variable_midpoints.size()[2],nb_of_hidden_layers=3, nb_of_hidden_nodes=25,batch_normalization=True)
my_net_directions=Net(x_variable.size()[1],y_variable_directions.size()[2],nb_of_hidden_layers=3, nb_of_hidden_nodes=25,batch_normalization=True)
optimizer_midpoints = torch.optim.Adam(my_net_midpoint.parameters(), lr=1e-4,weight_decay=0.5)
optimizer_directions = torch.optim.Adam(my_net_directions.parameters(), lr=1e-4,weight_decay=0.5)

torch.cuda.empty_cache()


In [ ]:
if  torch.cuda.is_available():
    #loss_func.cuda()
        
    x_variable , y_variable_midpoints,y_variable_directions=x_variable.cuda(), y_variable_midpoints.cuda(),y_variable_directions.cuda()
    x_variable_test,y_variable_midpoints_test= Variable(x_tensor_test.cuda(),volatile=True),Variable(y_tensor_midpoints_test.cuda(),volatile=True)
    y_variable_directions_test=Variable(y_tensor_directions_test.cuda(),volatile=True)
    #x_variable_test,y_variable_test= Variable(double_mixed_tensor_test.cuda(),volatile=True),Variable(y_tensor_test.cuda(),volatile=True)

    print("cuda activated")


In [ ]:
train_network(net=my_net_midpoint,loss_function=my_torch_loss_function,optimization=optimizer_midpoints,epochs=13000,variable_x=x_variable,variable_x_test=x_variable_test,
              variable_y=y_variable_midpoints,variable_y_test=y_variable_midpoints_test)



In [ ]:
train_network(net=my_net_directions,loss_function=my_torch_loss_function,optimization=optimizer_directions,epochs=13000,variable_x=x_variable,variable_x_test=x_variable_test,
                  variable_y=y_variable_directions,variable_y_test=y_variable_directions_test)


In [ ]:
# Visualization of error on the test data (not including random contours)
plt.clf()
sample_index=77
sample_contour=np.delete(polygons_reshaped[nb_of_training_data+sample_index],24)
#sample_contour=polygons_reshaped[sample_index]
sample_contour=sample_contour.reshape(12,2)
real_values=y_variable_test.cpu()
real_values=real_values.data.numpy()
inserted_point_coordinates=real_values[sample_index].reshape(nb_of_points,2)
inserted_point_coordinates=np.array(inserted_point_coordinates)


my_net.eval()
x_variable_test=x_variable_test.cuda()
predictions=my_net(x_variable_test).cpu()
predicted_inserted_points=predictions.data.numpy()[sample_index]
predicted_inserted_points=predicted_inserted_points.reshape(nb_of_points,2)


plt.scatter(predicted_inserted_points[:,0],predicted_inserted_points[:,1],label='FNN predicted point')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Point location')
#plt.scatter(polygons_reshaped[3112][0::2].sum()/12,polygons_reshaped[312][1::2].sum()/12,label='Original points')

plot_contour(sample_contour)
plt.legend()
plt.axis('scaled')
print('distance between points:',np.linalg.norm(predicted_inserted_points-inserted_point_coordinates))
#original_points=point_coordinates.reshape(218722,2)
#plt.scatter(original_points[:,0],original_points[:,1],label='Original points')
#plt.scatter(prediction[0],prediction[1])


In [ ]:
# Plotting the random contour using the FNN network  by predictiong both the midpoint and the direction#
plt.clf()
for i in range(1000):
    random_contour=apply_procrustes(generate_contour(12))
    random_nb_of_points,random_point_coordinated_delaunay=get_extrapoints_target_length_jupyter(random_contour,1,algorithm='del2d')
    if random_nb_of_points==nb_of_points:
            break
random_contour_reshaped=random_contour.reshape(1,2*12)
random_contour_with_target=np.hstack([random_contour_reshaped,[[1]]])



plot_contour(random_contour)
random_point_coordinated_delaunay=np.array(random_point_coordinated_delaunay)
random_point_coordinated_delaunay.reshape(nb_of_points,2)
#mid_point_delaunay=(random_point_coordinated_delaunay[0]+random_point_coordinated_delaunay[1])/2
#plt.scatter(mid_point_delaunay[0],mid_point_delaunay[1])
plt.scatter(random_point_coordinated_delaunay[:,0],random_point_coordinated_delaunay[:,1],label='Point location',marker='x')




random_x_variable=Variable(torch.from_numpy(random_contour_with_target))
random_x_variable=random_x_variable.expand(1000,2*12+1).type(torch.FloatTensor)
my_net=my_net.cpu()
my_net.eval()
random_prediction=my_net(random_x_variable)
random_prediction=random_prediction.data[0].numpy()
#middle_point,direction=np.array(random_prediction[:2]),np.array(random_prediction[2:])
#random_prediction=random_prediction.reshape(2,2)
points=random_prediction.reshape(nb_of_points,2)
#plt.scatter(middle_point[0],middle_point[1])
plt.scatter(points[:,0],points[:,1],label=' Prediction')
#plt.scatter(points[0,0]-0.05,points[0,1]+0.03,c='r',label=' CNN prediction')
#plt.scatter(points[1,0]-0.03,points[1,1]+0.01,c='r')


plt.legend()
#print(" Precision error: ",precision_error(random_point_coordinated_delaunay,points))
regression_error(random_point_coordinated_delaunay,points,nb_of_points)


In [ ]:
# Get a statistic on the regression loss

regression_errors=[]
for k in range(100):
    for i in range(1000):
        random_contour=apply_procrustes(generate_contour(12))
        random_nb_of_points,random_point_coordinated_delaunay=get_extrapoints_target_length_jupyter(random_contour,1,algorithm='del2d')
        if random_nb_of_points==nb_of_points:
                break
    random_contour_reshaped=random_contour.reshape(1,2*12)
    random_contour_with_target=np.hstack([random_contour_reshaped,[[1]]])
    random_point_coordinated_delaunay=np.array(random_point_coordinated_delaunay)
    random_point_coordinated_delaunay.reshape(nb_of_points,2)
    
    random_x_variable=Variable(torch.from_numpy(random_contour_with_target))
    random_x_variable=random_x_variable.expand(1000,2*12+1).type(torch.FloatTensor)
    my_net=my_net.cpu()
    my_net.eval()
    random_prediction=my_net(random_x_variable)
    random_prediction=random_prediction.data[0].numpy()
    points=random_prediction.reshape(nb_of_points,2)
    error=regression_error(random_point_coordinated_delaunay,points,nb_of_points)
    regression_errors.append(error)


regression_errors=np.array(regression_errors)
regression_errors.mean()

In [ ]:
regression_errors

In [ ]:
# Plotting random contour using 2 seperate jnn\s for the midpoint location and the direction  seperately#

#plt.clf()
#for i in range(1000):
#    random_contour=apply_procrustes(generate_contour(8))
#    random_nb_of_points,random_point_coordinated_delaunay=get_extrapoints_target_length_jupyter(random_contour,0.8,algorithm='del2d')
#    if random_nb_of_points==2:
#           break
random_contour_reshaped=random_contour.reshape(1,2*8)
random_contour_with_target=np.hstack([random_contour_reshaped,[[1]]])



plot_contour(random_contour)
random_point_coordinated_delaunay=np.array(random_point_coordinated_delaunay)
#random_point_coordinated_delaunay.reshape(2,2)
#mid_point_delaunay=(random_point_coordinated_delaunay[0]+random_point_coordinated_delaunay[1])/2
plt.scatter(random_point_coordinated_delaunay[:,0],random_point_coordinated_delaunay[:,1],label='Real point location',marker='x')




random_x_variable=Variable(torch.from_numpy(random_contour_with_target))
random_x_variable=random_x_variable.expand(1000,2*8+1).type(torch.FloatTensor)
my_net_midpoint,my_net_directions=my_net_midpoint.cpu(),my_net_directions.cpu()
my_net_midpoint.eval(),my_net_directions.eval()

random_midpoint_prediction=my_net_midpoint(random_x_variable)
random_midpoint_prediction=random_midpoint_prediction.data[0].numpy()


random_direction_prediction=my_net_directions(random_x_variable)
random_direction_prediction=random_direction_prediction.data[0].numpy()


points=np.array(get_points(random_midpoint_prediction,random_direction_prediction))
print(points[:,0]) 
plt.scatter(points[:,0],points[:,1],label='2 NN s prediction')
#plt.scatter(points[:,0],points[:,1],label='Prediction')
plt.legend()
print(" Precision error: ",precision_error(random_point_coordinated_delaunay,points))


In [ ]:
#plot_random_prediction_ffn(polygon_edges=8,net=my_net,number_of_points=2)

In [ ]:
random_contours=[]
target_edge_length=[]
random_point_coordinates=[]
for i in np.linspace(0.1,1,10):
    random_nb_of_points,random_point_coordinated_delaunay=get_extrapoints_target_length_jupyter(random_contour,i,algorithm='del2d')
    if random_nb_of_points==2:
            random_point_coordinates.append(random_point_coordinated_delaunay)
            random_contours.append(random_contour.reshape(1*16))
            target_edge_length.append(i)
            
            


In [ ]:
random_point_coordinates=np.array(random_point_coordinates)

In [ ]:
random_point_coordinates=random_point_coordinates.reshape(len(random_point_coordinates),1,4)

In [ ]:
contour_with_target_edge_length=[]
for index,contour in enumerate(random_contours):
    contour_with_target_edge_length.append(np.append(contour,target_edge_length[index]))
contour_with_target_edge_length=np.array(contour_with_target_edge_length)

In [ ]:
contour_with_target_edge_length.shape

In [ ]:
polygons_reshaped=np.vstack([polygons_reshaped,contour_with_target_edge_length])

In [ ]:
polygons_reshaped.shape

In [ ]:
point_coordinates=np.vstack([point_coordinates,random_point_coordinates])
point_coordinates.shape

In [ ]:
data_triangle=extract_lengths_angles_in_triangle_form(polygons_reshaped,12)
#data,lengths,angles=extract_lengths_angles(polygons_reshaped,12)




x_tensor_conv=torch.from_numpy(data_triangle[:nb_of_training_data]).type(torch.FloatTensor)
#mixed_tensor=torch.cat((lengths_tensor[:nb_of_training_data],angles_tensor[:nb_of_training_data]),1)
#double_mixed_tensor=angles_tensor[:nb_of_training_data]

x_tensor_test_conv=torch.from_numpy(data_triangle[nb_of_training_data:]).type(torch.FloatTensor)
#mixed_tensor_test=torch.cat((lengths_tensor[nb_of_training_data:],angles_tensor[nb_of_training_data:]),1)
#double_mixed_tensor_test=angles_tensor[nb_of_training_data:]




x_variable_conv,x_variable_test_conv=Variable(x_tensor_conv),Variable(x_tensor_test_conv)
#x_variable,x_variable_test=Variable(mixed_tensor),Variable(mixed_tensor_test)
#x_variable,x_variable_test=Variable(double_mixed_tensor),Variable(double_mixed_tensor_test)

#y_tensor=torch.from_numpy(mid_points_with_directions[:nb_of_training_data]).type(torch.FloatTensor)
#y_tensor_test=torch.from_numpy(mid_points_with_directions[nb_of_training_data:]).type(torch.FloatTensor)

#y_variable,y_variable_test=Variable(y_tensor),Variable(y_tensor_test)


y_tensor=torch.from_numpy(point_coordinates[:nb_of_training_data]).type(torch.FloatTensor)
y_tensor_test=torch.from_numpy(point_coordinates[nb_of_training_data:]).type(torch.FloatTensor)

y_variable,y_variable_test=Variable(y_tensor),Variable(y_tensor_test)

#shuffle=torch.randperm(x_variable.shape[0])
#x_variable_conv = x_variable_conv[shuffle]
#y_variable=y_variable[shuffle]
#print(y_variable.size(),x_variable.size())
#distances=[np.linalg.norm(i-j) for i in centers_of_mass for j in predicted_inserted_points]
#distances=np.array(distances)
#100*distances.sum()/(x_variable.size(0) *max_distance)

In [ ]:
# Reshape variables for convolutional neural net


x_variable_conv,x_variable_test_conv=x_variable_conv.resize(x_variable_conv.size()[0],1,x_variable_conv.size()[1]),Variable(x_tensor_test_conv.view(x_variable_test_conv.size()[0],1,x_variable_test_conv.size()[1]),volatile=True)

In [ ]:
torch.cuda.empty_cache()
#my_conv_net=my(nb_of_filters=10,out_dimension=2*nb_of_points,nb_of_hidden_nodes=15)
my_conv_net=triangle_convoluting_net(nb_of_filters=20,nb_of_hidden_nodes=63,out_dimension=2*nb_of_points,nb_of_edges=12)
print(my_conv_net)


In [ ]:
optimizer = torch.optim.Adam(my_conv_net.parameters(), lr=1e-4,weight_decay=0.5)

#optimizer = torch.optim.SGD(my_net.parameters(), lr=1e-5,weight_decay=.5,momentum=0.9)

loss_func = torch.nn.MSELoss(size_average=False) 
#loss_func=torch.nn.SmoothL1Loss()


In [ ]:
if  torch.cuda.is_available():
    loss_func.cuda()
    my_conv_net.cuda()
    x_variable_conv , y_variable,x_variable_test_conv,y_variable_test= x_variable_conv.cuda(), y_variable.cuda(),x_variable_test_conv.cuda(),y_variable_test.cuda()
    print("cuda activated")


In [ ]:
batch_size=int(x_variable_conv.size()[0])
nb_of_epochs=33000
# Train the network #
my_conv_net.train()
for t in range(nb_of_epochs):
    sum_loss=0
    for b in range(0,x_variable_conv.size(0),batch_size):
        out = my_conv_net(x_variable_conv.narrow(0,b,batch_size))                 # input x and predict based on x
        
        
       # target_edge_lengths=x_variable_conv.narrow(0,b,batch_size).resize(len(x_variable_conv.narrow(0,b,batch_size)),25)[:,24]
        
        
        loss = loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        
        #loss1=my_torch_loss_function(out, y_variable.narrow(0,b,batch_size).resize(batch_size,nb_of_points*2)).sum()
       # loss1=my_torch_loss_function3(out, y_variable.narrow(0,b,batch_size).resize(batch_size,nb_of_points*2),x_variable_conv.narrow(0,b,batch_size)[:,24],0.6).sum().cuda()

        #loss=loss_func.apply(out, y_variable.narrow(0,b,batch_size))
        #loss=torch.sqrt((out[0]-y_variable.narrow(0,b,batch_size)[0]).pow(2)+(out[1]-y_variable.narrow(0,b,batch_size)[1]).pow(2)) 
        sum_loss+=loss.data[0]
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        #print(t,loss.data[0])
        optimizer.step()        # apply gradients
    if t%10==0:
        my_conv_net.eval()
        out_test_conv=my_conv_net(x_variable_test_conv)
        #target_edge_lengths_test=x_variable_test_conv.resize(len(x_variable_test_conv),25)[:,24]
        test_loss=loss_func(out_test_conv,y_variable_test).data[0]
        #test_loss=my_torch_loss_function(my_conv_net(x_variable_test_conv),y_variable_test.resize(len(y_variable_test),2*nb_of_points)).sum().data[0]
        #test_loss=my_torch_loss_function3(out_test_conv, y_variable_test.resize(y_variable_test.size()[0],nb_of_points*2),x_variable_test[:,24],0.6).sum().cuda()

        #test_loss=my_torch_loss_function2(my_conv_net(x_variable_test_conv),y_variable_test.resize(len(y_variable_test),2*nb_of_points),target_edge_lengths_test).sum().data[0]
        my_conv_net.train()
        print("Epoch:",t,"Training Loss:",sum_loss/x_variable_conv.size(0),"Test Loss:",test_loss/x_variable_test_conv.size(0))

In [ ]:
plt.clf()
sample_index=85

contour1=np.delete(polygons_reshaped[nb_of_training_data+sample_index],2*12)
contour1=contour1.reshape(12,2)
#nb_of_inserted_points,inserted_point_coordinates=get_extrapoints_target_length_jupyter(contour1,.3,algorithm='del2d'
#                                                                              )
real_values=y_variable_test.cpu()
real_values=real_values.data.numpy()
inserted_point_coordinates=real_values[sample_index].reshape(12,2)
inserted_point_coordinates=np.array(inserted_point_coordinates)
my_conv_net.eval()
predictions=my_conv_net(x_variable_test_conv).cpu()
predicted_inserted_points=predictions.data.numpy()[sample_index]
predicted_inserted_points=predicted_inserted_points.reshape(12,2)
plt.scatter(predicted_inserted_points[:,0],predicted_inserted_points[:,1],label='CNN predicted points')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Point location')
plot_contour(contour1)
plt.legend()
plt.axis('scaled')

In [ ]:
x_variable_conv[1].narrow(0,1,3)

In [ ]:
predictions

In [ ]:
plot_random_prediction_cnn(8,my_conv_net,1)


In [ ]:
# Plot a prediction of a random contour based on a trained network (CNN)
#plt.clf()
#for i in range(1000):
#    random_contour=apply_procrustes(generate_contour(8))
#    random_nb_of_points,random_point_coordinated_delaunay=get_extrapoints_target_length_jupyter(random_contour,0.8,algorithm='del2d')
#    if random_nb_of_points==2:
#            break
random_contour_reshaped=random_contour.reshape(1,2*8)
random_contour_with_target=np.hstack([random_contour_reshaped,[[.8]]])
#triangles_with_edges=extract_lengths_angles_in_triangle_form(random_contour_with_target,8)
lengths_angles,lengths,angles=extract_lengths_angles(random_contour_with_target,8)


plot_contour(random_contour)
random_point_coordinated_delaunay=np.array(random_point_coordinated_delaunay)
random_point_coordinated_delaunay.reshape(2,2)
plt.scatter(random_point_coordinated_delaunay[:,0],random_point_coordinated_delaunay[:,1],label='Point location')




random_x_variable=Variable(torch.from_numpy(lengths_angles))
print(random_x_variable)
random_x_variable=random_x_variable.expand(1000,1,2*8+1).type(torch.FloatTensor)
my_conv_net=my_conv_net.cpu()
my_conv_net.eval()
random_prediction=my_conv_net(random_x_variable).data[0].numpy()
middle_point,direction=np.array(random_prediction[:2]),np.array(random_prediction[2:])
#random_prediction=random_prediction.reshape(2,2)
points=np.array(get_points(middle_point,direction))
plt.scatter(points[:,0],points[:,1],label='cnn prediction')
plt.legend()

In [ ]:
lengths_angles

In [ ]:
sample_index=150

contour1=np.delete(polygons_reshaped[nb_of_training_data+sample_index],2*8)
contour1=contour1.reshape(8,2)
#nb_of_inserted_points,inserted_point_coordinates=get_extrapoints_target_length_jupyter(contour1,.3,algorithm='del2d'
#                                                                              )
real_values=y_variable_test.cpu()
real_values=real_values.data.numpy()
inserted_point_coordinates=real_values[sample_index].reshape(2,2)
inserted_point_coordinates=np.array(inserted_point_coordinates)
my_conv_net.eval()
predictions=my_conv_net(x_variable_test_conv).cpu()
predicted_inserted_points=predictions.data.numpy()[sample_index]
predicted_inserted_points=predicted_inserted_points.reshape(2,2)
plt.scatter(predicted_inserted_points[:,0],predicted_inserted_points[:,1],label='CNN predicted points')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Point location')
plot_contour(contour1)
plt.legend()
plt.axis('scaled')


In [ ]:
plot_random_prediction_cnn(polygon_edges=8,net=my_conv_net,number_of_points=2)

In [ ]:
pertubed_contours=np.array(pertubed_contours)
pertubed_contours=pertubed_contours.reshape(len(pertubed_contours),8*2)
edge_lengths=np.array(edge_lengths)
edge_lengths=edge_lengths.reshape(len(edge_lengths),1)

In [ ]:
pertubed_contours_with_edge_lengths=np.hstack([pertubed_contours,edge_lengths])

In [ ]:
pertubed_contours_with_edge_lengths

In [ ]:
 a1,d1,g1=generate_data(my_net,x_variable_test,0.2)

In [ ]:
contours_pertubed=np.array(a)
contours_pertubed=contours_pertubed.reshape(len(contours_pertubed),16)
edge_lengths=np.array(d)
edge_lengths=edge_lengths.reshape(len(edge_lengths),1)

contour_pertubed_with_edge_lengths=np.hstack([contours_pertubed,edge_lengths])
point_coordinates_of_pertubed_contours=np.array(d)
point_coordinates_of_pertubed_contouurs=point_coordinates_of_pertubed_contours.reshape(len(point_coordinates_of_pertubed_contours),2)

In [ ]:
contours_pertubed1=np.array(a1)
contours_pertubed1=contours_pertubed1.reshape(len(contours_pertubed1),16)
edge_lengths1=np.array(d1)
edge_lengths1=edge_lengths1.reshape(len(edge_lengths1),1)

contour_pertubed_with_edge_lengths1=np.hstack([contours_pertubed1,edge_lengths1])
point_coordinates_of_pertubed_contours1=np.array(g1)
point_coordinates_of_pertubed_contours1=point_coordinates_of_pertubed_contours1.reshape(len(point_coordinates_of_pertubed_contours1),2)

In [ ]:
pre_con1,edgm1,sdf1=generate_data(my_net,x_variable_test,tol=0.2)


In [ ]:
contours_pertubed3=np.array(pre_con1)
contours_pertubed3=contours_pertubed3.reshape(len(contours_pertubed3),16)
edge_lengths3=np.array(edgm1)
edge_lengths3=edge_lengths3.reshape(len(edge_lengths3),1)

contour_pertubed_with_edge_lengths3=np.hstack([contours_pertubed3,edge_lengths3])
point_coordinates_of_pertubed_contours3=np.array(sdf1)
point_coordinates_of_pertubed_contours3=point_coordinates_of_pertubed_contours3.reshape(len(point_coordinates_of_pertubed_contours3),2)

In [ ]:
contour_pertubed_with_edge_lengths3.shape

In [ ]:
 np.unique(pre_con_array)

In [ ]:
'''
Should I order the interior points ?

For each interior point calculate distance with points of the polygon

find which ti which point it is closer and sort it according to the index.

If multiple points are closer to the same point of the contour then take into

account the distance to the point.

'''


In [ ]:
perturbed_contour_1,edge_lengths_1,points_coordinates_del_1=generate_data_for_target_edge_length(my_net,x_variable_test,0.2,1)

In [ ]:
contours_pertubed_1=np.array(perturbed_contour_1)
contours_pertubed_1=contours_pertubed_1.reshape(len(contours_pertubed_1),16)
edge_lengths_1=np.array(edge_lengths_1)
edge_lengths_1=edge_lengths_1.reshape(len(edge_lengths_1),1)

contour_pertubed_with_edge_lengths_1=np.hstack([contours_pertubed_1,edge_lengths_1])
point_coordinates_of_pertubed_contours_1=np.array(points_coordinates_del_1)
point_coordinates_of_pertubed_contours_1=point_coordinates_of_pertubed_contours_1.reshape(len(point_coordinates_of_pertubed_contours_1),2)

In [ ]:
pre_con1

In [ ]:
for polygon in pre_con1[1:400]:
    plot_contour(polygon)

In [ ]:
extract_lengths_angles_in_triangle_form(random_contour.reshape(1,16),len(random_contour))

In [ ]:
random_contour

In [ ]:
plt.plot([1,2,3])
# now create a subplot which represents the top plot of a grid
# with 2 rows and 1 column. Since this subplot will overlap the
# first, the plot (and its axes) previously created, will be removed
plt.subplot(222)
plt.plot(range(12))
plt.subplot(222, facecolor='y') # creates 2nd subplot with yellow background

In [ ]:
y_variable_test=y_variable_test.cpu().resize(y_variable_test.size()[0],y_variable_test.size()[2])

In [ ]:

plot_contour(contour1)
plt.scatter(predictions.data.numpy()[:,0],predictions.data.numpy()[:,1])
plt.scatter(y_variable_test.data.numpy()[:,0],y_variable_test.data.numpy()[:,1])



In [ ]:
extract_lengths_angles(random_contour_with_target,8)

In [ ]:
random_contour_with_target

In [ ]:
points=point_coordinates.reshape(len(point_coordinates),2,2)

In [ ]:
points[3]

In [ ]:
get_points(mid_points[3],directions[3])

In [ ]:
plt.scatter(points[13][0][0],points[13][0][1])
plt.scatter(points[13][1][0],points[13][1][1])
plt.scatter(mid_points[13][0],mid_points[13][1])
plt.quiver(mid_points[13][0],mid_points[13][1],points[13][0][0],points[13][0][1],scale_units='xy')
plt.axis('scaled')

In [ ]:
rotation_projection

In [ ]:
# Main Idea for positioning index: For  a set of n interior point calculate the braycenter of the contour that is 
# formed by connnectiong the points.  For each edges of the interior point polygon connect it with the barycenter
# to form a tricngle. Xw=( c-x_1,c-x_2) ->  F= XwXm*(-1) -> X=C+ FXm. F contains information about the rotation and trans-
# ation. We have to keep in mind have fixed axis of the orthogonal eigenvectors.
# If u_x <0 then u=-u and v=-v. If w_z<0 (3D)  tjem v=-v and w=-w

# mid_points[13][0]

In [ ]:
pi

In [ ]:
extract_midpoint_directions(point_coordinates)

In [ ]:
point_coordinates[0]

In [ ]:
mid_point_with_direction=extract_midpoint_directions([point_coordinates[0]])

In [ ]:
mid_point=np.array(mid_point_with_direction[0])
direction=np.array(mid_point_with_direction[])

In [ ]:
get_points(mid_point,direction)

In [ ]:
mid_point=mid_point_with_direction[0][:2]
direction=mid_point_with_direction[0][2:]

In [ ]:
get_points(mid_point,direction)

In [ ]:
point_coordinates

In [ ]:
out_test

In [ ]:
y_tensor_test

In [ ]:
                                # Project set of polygons in 2d and 3d space #
                        # Dimensionality reduction using Isomap, PCA, kernel PCA ... #

from sklearn.decomposition import PCA,KernelPCA






                        # Isomap #
#nb_components=8
#iso=manifold.Isomap(n_neighbors=8,n_components=nb_components,n_jobs=-1)
#iso.fit(Polygons_reshaped)
#Polygons_projected=iso.transform(Polygons_reshaped)


                        # PCA   #
pca=PCA(.7)
pca.fit(point_coordinates_reshaped)
point_coordinates_projected=pca.transform(point_coordinates_reshaped)
nb_components=int(pca.n_components_)

print(nb_components)
# Fitting into lesser dimension

#Polygons_projected=iso.fit_transform(Polygons_reshaped)
#Polygons_projected=iso.transform(Polygons_reshaped)

#iso = KernelPCA(n_components=2,kernel="rbf", fit_inverse_transform=True, gamma=1e-1)



plt.scatter(Polygons_projected[:,0],Polygons_projected[:,1])



In [ ]:
point_coordinates_reshaped= point_coordinates.reshape(len(point_coordinates),4)

In [ ]:
precision_error(points,random_point_coordinated_delaunay)

In [ ]:
np.linalg.norm(points[0]- random_point_coordinated_delaunay[0])

In [ ]:
points[0]
random_point_coordinated_delaunay[0]

In [ ]:
max_element(distances)

In [ ]:
test_target_edge_lengths=x_variable_test.data.numpy()[:,16]

In [ ]:
real_midpoints,real_directions=y_tensor_test.numpy().reshape(len(y_tensor_test),4)[:,0:2],y_tensor_test.numpy().reshape(len(y_tensor_test),4)[:,2:4]

real_points=[]
for index,_ in enumerate(real_directions):
        real_points.append(get_points(real_midpoints[index],real_directions[index]))
    

x_variable_test=x_variable_test.cpu()
my_net=my_net.cpu()
predicted_midpoints,predicted_directions=my_net(x_variable_test).data.numpy()[:,0:2],my_net(x_variable_test).data.numpy()[:,2:4]

predicted_points=[]
for index,_ in enumerate(real_directions):
            predicted_points.append(get_points(predicted_midpoints[index],predicted_directions[index]))

precision_errors=np.empty(len(predicted_midpoints))
real_points=np.array(real_points)
predicted_points=np.array(predicted_points)
for index,_ in enumerate(predicted_midpoints):
    error=np.divide(precision_error(real_points[index],predicted_points[index]),test_target_edge_lengths[index])
    #error=precision_error(real_points[index],predicted_points[index])
    precision_errors[index]=error

precision_errors.mean()

In [ ]:
my_net_midpoint=my_net_midpoint.cpu()
my_net_directions=my_net_directions.cpu()
predicted_midpoints2=my_net_midpoint(x_variable_test).data.numpy()
predicted_directions2=my_net_directions(x_variable_test).data.numpy()

predicted_points2=[]
for index,_ in enumerate(real_directions):
            predicted_points2.append(get_points(predicted_midpoints2[index],predicted_directions2[index]))

precision_errors=np.empty(len(predicted_midpoints))
real_points=np.array(real_points)
predicted_points=np.array(predicted_points)
for index,_ in enumerate(predicted_midpoints):
    error=np.divide(precision_error(real_points[index],predicted_points2[index]),test_target_edge_lengths[index])
    #error=precision_error(real_points[index],predicted_points2[index])
    precision_errors[index]=error

precision_errors.mean()